# Instalación de paquetes

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.1 MB/s et

Necesitará configurar git, adaptar su correo electrónico y nombre en la siguiente celda.

In [ ]:
!git config --global user.email "paul.rojas@correounivalle.edu.co"
!git config --global user.name "paulrojasg"

Primero debe almacenar su token de autenticación del sitio web de Hugging Face (¡regístrese aquí si aún no lo ha hecho!), luego ejecute la siguiente celda e ingrese su nombre de usuario y contraseña:

In [2]:
## PUNTO 2.1 ## Primero se debe almacenar su token de autenticación del sitio web de Hugging Face
from huggingface_hub import notebook_login

notebook_login()

## Importaremos CONLL2002

In [3]:
## PUNTO 2.2 ## Cargar el dataset de conll2002 desde HF, realizar las pruebas del dataset,
# conjunto train, test y val, además de verificar el nombre de la etiquetas de
# anotado.
from datasets import load_dataset

raw_datasets = load_dataset("conll2002", 'es')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for conll2002 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2002
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/8324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1518 [00:00<?, ? examples/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1518
    })
})

Podemos acceder a un elemento real, primero debe seleccionar una división y luego proporcionar un índice:

Entonces, para las etiquetas NER, 0 corresponde a 'O', 1 a 'B-PER', etc... Además de la 'O' (que significa que no hay entidad especial), aquí hay cuatro etiquetas para NER, cada una con el prefijo 'B-' (para inicio) o 'I-' (para intermedio), que indican si el token es el primero del grupo actual con la etiqueta o no:

'PER' para persona
'ORG' para organización
'LOC' para ubicación
'MISC' para varios

In [5]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

Dado que las etiquetas son listas de ClassLabel, los nombres reales de las etiquetas están anidados con el atributo de característica del objeto anterior:

In [6]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [7]:
print(raw_datasets["train"][0]["tokens"])
print(raw_datasets["train"][0]["ner_tags"])

['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.']
[5, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0]


## **1. Preprocesamiento del dataset**
Antes de que podamos alimentar esos textos a nuestro modelo, debemos preprocesarlos. Esto se hace mediante un 🤗 Transformers Tokenizer que (como su nombre indica) tokenizará las entradas (incluida la conversión de los tokens a sus ID correspondientes en el vocabulario previamente entrenado) y las colocará en el formato que espera el modelo, además de generar las otras entradas que el  modelo requiere.

Para hacer todo esto, creamos una instancia de nuestro tokenizador con el método **AutoTokenizer.from_pretrained**, que garantizará:

- Obtenemos un tokenizador que corresponde a la arquitectura del modelo que queremos usar,
- Descargamos el vocabulario utilizado al entrenar previamente este punto de control específico.

Ese vocabulario se almacenará en caché, por lo que no se volverá a descargar la próxima vez que ejecutemos la celda.

In [8]:
## PUNTO 2.3.a ## Implementar la fase de preprocesamiento definiendo el tokenizador del
# modelo de bert-base-cased, alinear los tokens y etiquetas con los
# tokenizadores de bert-base-cased

# This uses a tokenizer pre-trained based on bert-base-cased model
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

La siguiente afirmación garantiza que nuestro tokenizador es un tokenizador rápido (respaldado por Rust) de la biblioteca 🤗 Tokenizers. Esos tokenizadores rápidos están disponibles para casi todos los modelos y necesitaremos algunas de las características especiales que tienen para nuestro preprocesamiento.

In [9]:
tokenizer.is_fast

True



Aquí las  entradas ya se han dividido en palabras y se  deben pasar la lista de palabras a su tokenzier con el argumento **is_split_into_words=True:**

In [10]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Melbourne',
 '(',
 'Australia',
 ')',
 ',',
 '25',
 'may',
 '(',
 'E',
 '##F',
 '##E',
 ')',
 '.',
 '[SEP]']

In [11]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 8, 9, 10, None]

Aquí configuramos las etiquetas de todos los tokens especiales (CLS y SEP) en -100 (el índice que PyTorch ignora) y las etiquetas de todos los demás tokens en la etiqueta de la palabra de la sentencia que provienen. Por cada sentencia del dataset se cololan [-100-----100] por [[CLS].....[SEP]].Este procedimiento también realiza la alineación de los tokens de las sentencias con las etiquetas y sus tokens especiales.

In [12]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

Este es un ejemplo de como una sentencia de tags enterizada es alineada.

In [13]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(word_ids)

print(align_labels_with_tokens(labels, word_ids))

[5, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0]
[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 8, 9, 10, None]
[-100, 5, 0, 5, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, -100]


In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

En esta parte se el tokenizador del transformer (wordpiece) alinea los tokens de las sentencias de entreanamiento (5 en este caso). La alineación consiste en que el tokenizador 1) Enteriza con  **input_ids** todos los tokens de las 5 sentencias, aquí no hay padding. 2) Tokinizer asigna los valores de token_type_ids y attention_mask para las 5 senetncias. 3) Se realiza el alineamiento con los símbolos especiales  ([CLS], [SEP]) que valdrían -100.

In [15]:
tokenize_and_align_labels(raw_datasets['train'][:5])

{'input_ids': [[101, 4141, 113, 1754, 114, 117, 1512, 1336, 113, 142, 2271, 2036, 114, 119, 102], [101, 118, 102], [101, 2896, 138, 4043, 23224, 1615, 3687, 142, 18735, 1186, 117, 25026, 2902, 117, 4841, 6447, 7774, 16358, 1183, 2495, 24928, 7723, 6859, 1181, 1260, 1106, 7317, 1143, 3309, 9028, 18311, 5250, 1566, 2895, 2393, 27466, 13894, 1918, 9799, 12686, 16468, 15647, 1186, 175, 17759, 1162, 170, 8362, 1161, 185, 5589, 10533, 1161, 1260, 7210, 15027, 24034, 2155, 21883, 12888, 1161, 8468, 16040, 1643, 24891, 9080, 1186, 1260, 12724, 185, 4854, 6617, 27931, 1116, 171, 10014, 10658, 1116, 1260, 2495, 3180, 1183, 119, 102], [101, 2001, 11109, 27989, 4722, 3687, 138, 4043, 23224, 1615, 5069, 1673, 181, 15650, 1197, 3532, 16091, 10051, 1260, 15027, 8362, 179, 4175, 1584, 3687, 17632, 15463, 1643, 16996, 1186, 3687, 12890, 9359, 1260, 3006, 113, 1754, 114, 14516, 191, 25579, 1111, 24981, 1186, 170, 4267, 24313, 4121, 8362, 179, 4084, 2572, 1927, 194, 28117, 20080, 15981, 8468, 5250, 7723,


En esta parte se tokeniza y se alinean todas las etiquetas del dataset con el tokenizador de BERT que es wordpiece. Para aplicar esta función en todas las oraciones del dataset, simplemente usamos el método de mapeo  **.map**. Esto aplicará la función en todos los elementos de todas las divisiones del dataset; al conjunto de  entrenamiento, validación y testeo.

In [16]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)


Map:   0%|          | 0/8324 [00:00<?, ? examples/s]

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

In [17]:
## PUNTO 2.3.b ## En esta fase también se debe
# realizar el padding o relleno de manera dinámica usando la librería
# DataCollatorForTokenClassification.
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    5,    0,    5,    0,    0,    0,    0,    0,    3,    4,    4,
            0,    0, -100],
        [-100,    0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100]])

In [19]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 5, 0, 5, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, -100]
[-100, 0, -100]


In [20]:
# Seqeval is used for evaluating sequential labeling tasks
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=71122d414648e7b8bc187edb135ae1d8d489d14e0ee4af643b3e3d1129e9a754
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [21]:
import evaluate

metric = evaluate.load("seqeval")

In [22]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']

In [23]:
#predictions = labels.copy()
#predictions[2] = "O"
#metric.compute(predictions=[predictions], references=[labels])

In [24]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

## **2. Finetuning del modelo**

Ahora que nuestros datos están listos, podemos descargar el modelo previamente entrenado y ajustarlo. En este caso se usa el preentrenado **model_checkpoit="bert-base-cased"**  Dado que todas nuestras tareas tienen que ver con la clasificación de tokens, utilizamos la clase **AutoModelForTokenClassification**. Al igual que con el tokenizador, el método **from_pretrained** descargará y almacenará en caché el modelo por nosotros. Lo único que tenemos que especificar es el número de etiquetas para nuestro problema.

In [25]:
## PUNTO 2.4 ## Implementar el ajuste del modelo usando la tarea de clasificación del modelo
# preentrenado para la clasificación de entidades, para ello se usa la librería
# AutoModelForTokenClassification.
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model.config.num_labels

9



La siguiente función realiza todo este posprocesamiento del resultado de Trainer.evaluate (que es una tupla con nombre que contiene predicciones y etiquetas) antes de aplicar la métrica:

# **3 Entrenamiento del modelo**

Para crear una instancia de un **Trainer**, necesitaremos definir tres cosas más. El más importante es **TrainingArguments**, que es una clase que contiene todos los atributos para personalizar el entrenamiento. Requiere un nombre de carpeta, en este caso, **bert-finetuned-ner** que se utilizará para guardar los puntos de control del modelo, y todos los demás argumentos son opcionales. Aquí en esta parte se crea el batch y se define el número  de épocas, configuramos la evaluación que se realizará al final de cada época, ajustamos la tasa de aprendizaje, usamos el batchsize y personalizamos la cantidad de épocas para el entrenamiento, así como la disminución del peso. push_to-hub crea un modelo eb su hub.


In [27]:
## PUNTO 2.5 ## Definir la estrategia de entrenamiento usando TrainingArguments, en la que se
# definen el nombre del modelo con el que va quedar registrado en HF, el número de
# épocas, la estrategia de evaluación, la tolerancia o rata de aprendizaje.
from transformers import TrainingArguments

repositoryName = "bert-finetuned-ner-1"
args = TrainingArguments(
    repositoryName,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
    #batch_eval_metrics=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8
)

El Trainer es el correspondiente al fit  en keras que define el conjunto propio de validación en batch a través del data_collator y también se calcula el f1-score por cada época.

In [28]:
## PUNTO 2.6 ## Implementar el Trainer o fit del modelo en la que se deben definir el conjunto de
# entrenamiento, el de validación y poner en marcha el entrenamiento del modelo
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
  true_predictions = [
      [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
  return {
      "precision": all_metrics["overall_precision"],
      "recall": all_metrics["overall_recall"],
      "f1": all_metrics["overall_f1"],
      "accuracy": all_metrics["overall_accuracy"]
  }

In [29]:

from transformers import Trainer

#2.7 En la fase de entrenamiento se debe llamar un procedimiento que obtenga las
#métricas de desempeño de precisión, recall y f1-score y accuracy de cada una de
#las épocas para cada uno de los dos modelos. Esto se debe ver reflejado en la API
#de Hugging Face

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


# Publicación de modelo HF


In [ ]:
from huggingface_hub import ModelCard, whoami

metrics = trainer.state.log_history

training_details = {
    "epochs": args.num_train_epochs,
    "learning_rate": args.learning_rate,
    "weight_decay": args.weight_decay,
    "train_batch_size": args.train_batch_size,
    "eval_batch_size": args.eval_batch_size
}

metrics_data = "| Epoch | Training Loss | Validation Loss | Precision | Recall | F1 Score | Accuracy |\n"
metrics_data += "|-------|---------------|-----------------|-----------|--------|----------|----------|\n"

for entry in metrics:
    if 'loss' in entry:
        epoch = round(entry['epoch'])
        train_loss = entry['loss']
        metrics_data += f"| {epoch} | {train_loss:.4f}"

    elif 'eval_loss' in entry:
      val_loss = entry['eval_loss']
      precision = entry.get('eval_precision', 'N/A')
      recall = entry.get('eval_recall', 'N/A')
      f1 = entry.get('eval_f1', 'N/A')
      accuracy = entry.get('eval_accuracy', 'N/A')
      metrics_data += f"| {val_loss:.4f} | {precision:.4f} | {recall:.4f} | {f1:.4f} | {accuracy:.4f} |\n"


print(metrics_data)

card_content = f"""
# Model Name: {args.repositoryName}

This is a BERT model fine-tuned for Named Entity Recognition (NER).

## Training Details

- Epochs: {training_details['epochs']}
- Learning Rate: {training_details['learning_rate']}
- Weight Decay: {training_details['weight_decay']}
- Batch Size (Train): {training_details['train_batch_size']}
- Batch Size (Eval): {training_details['eval_batch_size']}

## Training Metrics

{metrics_data}
"""

model_card = ModelCard(card_content)

user = whoami()['name']

model_card.push_to_hub(repo_id=f'{user}/{repositoryName}')